In [1]:
import numpy as np
import pandas as pd
import json
import requests
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core import Workspace, Dataset


In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='NASA TurboFan AutoML')
df_pd = dataset.to_pandas_dataframe()

Getting data access token with Assigned Identity (client_id=clientid).


In [4]:
webservice_name = 'turbofan-rul-service'

In [5]:
service = Webservice(ws, webservice_name) 

uri = service.scoring_uri
key =service.get_keys()[0]

print(uri)

sampleSize = round(4/len(df_pd),4)
testdf = df_pd.sample(frac=sampleSize)

X = testdf.drop('rul',axis=1)
y = testdf.rul.tolist()


input_data = json.dumps(X.to_json( orient='records')) # Create JSON of sample records    

headers = {'Content-Type':'application/json'}

#for AKS deployment you'd need to the service key in the header as well    
headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ key)} 

resp = requests.post(uri, input_data, headers=headers)
try:
    print("POST to url", uri)
    print("input data:", input_data)
    print("label:", y)
    print("prediction:", resp.text)
except Exception:
    print("Web Service Not Found")

http://4b50fc71-d9f3-4cec-b24c-bca46939b03e.southcentralus.azurecontainer.io/score
POST to url http://4b50fc71-d9f3-4cec-b24c-bca46939b03e.southcentralus.azurecontainer.io/score
input data: "[{\"unit\":8,\"cycle\":32,\"os1\":0.0001,\"os2\":0.0002,\"os3\":100.0,\"sm1\":518.67,\"sm2\":642.87,\"sm3\":1594.16,\"sm4\":1413.85,\"sm5\":14.62,\"sm6\":21.61,\"sm7\":552.92,\"sm8\":2388.13,\"sm9\":9049.25,\"sm10\":1.3,\"sm11\":47.42,\"sm12\":521.58,\"sm13\":2388.17,\"sm14\":8125.78,\"sm15\":8.4686,\"sm16\":0.03,\"sm17\":394,\"sm18\":2388,\"sm19\":100.0,\"sm20\":38.73,\"sm21\":23.2709},{\"unit\":78,\"cycle\":117,\"os1\":0.001,\"os2\":0.0004,\"os3\":100.0,\"sm1\":518.67,\"sm2\":641.81,\"sm3\":1585.26,\"sm4\":1396.06,\"sm5\":14.62,\"sm6\":21.61,\"sm7\":554.94,\"sm8\":2388.05,\"sm9\":9059.54,\"sm10\":1.3,\"sm11\":47.38,\"sm12\":521.83,\"sm13\":2388.01,\"sm14\":8140.81,\"sm15\":8.3748,\"sm16\":0.03,\"sm17\":389,\"sm18\":2388,\"sm19\":100.0,\"sm20\":39.12,\"sm21\":23.3804},{\"unit\":37,\"cycle\":4,\"os